## Boosting e XGBoost (aula02 e aula03)

A metodologia  foi criada, inicialmente, para resolver uma classificação. A ideia principal é encontrar hipóteses fracas, aprender repetidamente e combinar essas hipóteses fracas dentro de uma única hipótese.
É considerada um método de ensemble. O método Ensemble tem como objetivo **combinar as predições de diversos estimadores mais simples** para gerar uma predição final mais robusta.
- Métodos de Boosting têm como procedimento geral a construção de estimadores de forma sequencial, de modo que estimadores posteriores tentam reduzir o viés do estimador conjunto, que leva em consideração estimadores anteriores. Ex.: **adaboost**.

### Método de Ensemble

Os métodos de Ensemble que tem como objetivo **combinar as predições de diversos estimadores mais simples** para gerar uma **predição final mais robusta**. Os métodos de ensemble costumam ser divididas em duas classes:
- **Métodos de média:** têm como procedimento geral construir diversos estimadores independentes, e tomar a média de suas predições como a predição final. O principal objetivo do método é reduzir **variância**, de modo que o modelo final seja melhor que todos os modelos individuais. Ex.: RnadomForest
- **Métodos de boosting:** têm como procedimento geral a construção de estimadores de forma sequencial, de modo que estimadores posteriores tentam reduizr o **viés** do estimador conjunto, que leva em consideração estimadores anteriores. Ex.: adaboost. 

### Bagging vs Boosting

- **Bagging**: Paralelo
- **Boosting**: Sequencial

### AdaBoost

Significa **Adaptative Boosting**, e tem como procedimento geral **a criação sucessiva dos chamados weak learners**, que são models bem fracos de aprendizagem - geralmente, **árvores de um único nó (stumps)**

<img src="https://miro.medium.com/max/1744/1*nJ5VrsiS1yaOR77d4h8gyw.png" width=300>

O AdaBoost utiliza os erros da árvore anterior para melhorar a próxima árvore. As predições finais são feitas com base nos pesos de cada stump, cusa determinação faz parte do algoritmo.

Aqui, o bootstrapping não é utilizado. O método começa treinando um classificador fraco no dataset original, e depois treina diversas cópias adicionais do classificador no mesmo dataset, mas dando um peso maior às observações que foram classificados erroneamente (ou, no caso de regressões, a observações **com o maior erro**).

Assim, após diversas iterações, classificadores / regressores vão sequencialmente "focando nos casos mais difíceis", e construindo um classificador encadeado que seja forte, apesar de utilizar diversos classificadores fracos como elementos fundamentais.

Único hiperparâmetro é n_estimators que é o número de weak learners encadeados.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('./datasets/german_credit_data.csv', index_col=0)
df.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NaN,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad


In [4]:
# Train_test_split
X = df.drop(columns = ["Risk"])
y = df["Risk"]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=y)

In [5]:
# Separação de tipos de variáveis
num_cols = X_train.select_dtypes("number").columns
cat_cols = X_train.select_dtypes(exclude = "number").columns

In [6]:
# Preprocessamento

# Impute | Transform
num_features = Pipeline([
    ("imputer_num", SimpleImputer(strategy = 'mean')),
    ("scaler", StandardScaler())
])

# Impute | Transform
cat_features = Pipeline([
    ("imputer_cat", SimpleImputer(strategy = 'constant', fill_value='unknown')),
    ("ohe", OneHotEncoder())
])

# preprocessamento do Impute | Transform
preprocessor = ColumnTransformer([
    ("transf_num", num_features, num_cols),
    ("transf_cat", cat_features, cat_cols)
])

# Rodar o modelo
pipe = Pipeline([
    ("preprocessor", preprocessor),
    ("model", AdaBoostClassifier(random_state = 42))
])

In [7]:
# Fitar o pipe
pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('transf_num',
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Age', 'Job', 'Credit amount', 'Duration'], dtype='object')),
                                                 ('transf_cat',
                                                  Pipeline(steps=[('imputer_cat',
                                                                   SimpleImputer(fill_value='unknown',
                                                                                 strategy='constant')),
                                                                  ('ohe',
                         

In [8]:
# Predict
y_pred = pipe.predict(X_test)

In [10]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         bad       0.62      0.48      0.54        60
        good       0.80      0.87      0.83       140

    accuracy                           0.76       200
   macro avg       0.71      0.68      0.69       200
weighted avg       0.74      0.76      0.75       200



### Gradient boosting

Existem outras classes de ensemble, em particular uma que usa o **gradient boosting**, que é baseado na utilização de weak learners sequencialmente adicionados de modo a **sequencialmente minimizar os erros cometidos**.

O objetivo geral do método é bem simples: determinar quais são os **parâmetros** da hipótese que minimizam a função de custo/perda. Para isso, o método "percorre" a função de erro, indo em direção ao seu mínimo (este caminho feito na função se dá justamente pela **determinação iterativa dos parâmetros**, isto é, **a cada passo, chegamos mais perto dos parâmetros finais da hipótese**, conforme eles são ajustados aos dados.)

Os principais hiperparâmetros a serem ajustados são:
- n_estimators: novamente, o número de weak learners encadeados
- learning_rate: a constante que multiplica o gradiente no fradiente descendente. Essencialmente, controla o "tamanho do passo" a ser dado em direção ao mínimo (recomendado <= 0.1).

In [12]:
from sklearn.ensemble import GradientBoostingClassifier

pipe_gb = Pipeline([
    ("preprocessor", preprocessor),
    ("gb_model", GradientBoostingClassifier(random_state=42))
])

pipe_gb.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('transf_num',
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Age', 'Job', 'Credit amount', 'Duration'], dtype='object')),
                                                 ('transf_cat',
                                                  Pipeline(steps=[('imputer_cat',
                                                                   SimpleImputer(fill_value='unknown',
                                                                                 strategy='constant')),
                                                                  ('ohe',
                         

In [13]:
y_pred = pipe.predict(X_test)

In [14]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         bad       0.62      0.48      0.54        60
        good       0.80      0.87      0.83       140

    accuracy                           0.76       200
   macro avg       0.71      0.68      0.69       200
weighted avg       0.74      0.76      0.75       200



----

### XGBoost

é o último modelo de Ensemble. Este método é um gradient boosting, com modificações que deixam mais 'extreme'.
- Adição de regularização (L1 e L2) que melhora a capacidade de generalização
- Usa derivadas de segunda ordem para o gradiente
- A construção sequencial é paralelizada
- Poda da árvore
- Otimização de hardware
- Regulariza por Lasso e Ridge
- Trata valores faltantes, já "aprende" qual o melhor valor para adotar
- Aplica quantil ponderado nos dados
- Validação cruzada integrado

Ref:
https://medium.com/analytics-vidhya/what-makes-xgboost-so-extreme-e1544a4433bb
https://xgboost.readthedocs.io/en/latest/tutorials/model.html

In [20]:
from xgboost import XGBClassifier

In [24]:
df = pd.read_csv('datasets/german_credit_data.csv', index_col=0)
df.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NaN,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad


In [84]:
# Separar features de target
X = df.drop(columns = ["Risk"])
y = df["Risk"]

In [85]:
# train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.2)

In [86]:
# Coletar colunas numéricas e categóricas
num_cols = X_train.select_dtypes('number').columns
cat_cols = X_train.select_dtypes(exclude = 'number').columns

In [87]:

cat_features = Pipeline([
    ("ohe", OneHotEncoder())
])
preprocessor = ColumnTransformer([
    ("transf_cat", cat_features, cat_cols)
])

pipe_xgb = Pipeline([
    ("preprocessor", preprocessor),
    ("xgb", XGBClassifier(learning_rate = 0.1, eval_metric = "logloss"))
])

In [88]:
pipe_xgb.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('transf_cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder())]),
                                                  Index(['Sex', 'Housing', 'Saving accounts', 'Checking account', 'Purpose'], dtype='object'))])),
                ('xgb',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eval_metric='logloss',
                               gamma=0, gpu_id=-1, importance_type='gain',
                               interaction_constraints='', learning_rate=0.1,
                               max_delta_step=0, max_depth=6,
                               min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
     

In [89]:
y_pred_train = pipe_xgb.predict(X_train)
y_pred_test = pipe_xgb.predict(X_test)

In [90]:
print(classification_report(y_train, y_pred_train))
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

         bad       0.84      0.72      0.78        58
        good       0.89      0.94      0.92       142

    accuracy                           0.88       200
   macro avg       0.87      0.83      0.85       200
weighted avg       0.88      0.88      0.88       200

              precision    recall  f1-score   support

         bad       0.45      0.38      0.41       242
        good       0.75      0.80      0.77       558

    accuracy                           0.67       800
   macro avg       0.60      0.59      0.59       800
weighted avg       0.66      0.67      0.66       800

